In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

100%|██████████| 49.4M/49.4M [00:04<00:00, 11.7MB/s]


In [2]:
tracker=Tracker()
count=0

In [3]:
cap=cv2.VideoCapture('traffictrim.mp4')

down={}
counter_down=set()
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    #print(px)
    
    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])


    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        # cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw ceter points of bounding box
        # cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        # cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



        y = 308
        offset = 7
    
        ''' condition for red line '''
        if y < (cy + offset) and y > (cy - offset):
          ''' this if condition is putting the id and the circle on the object when the center of the object touched the red line.'''
          
          down[id]=cy   #cy is current position. saving the ids of the cars which are touching the red line first. 
          #This will tell us the travelling direction of the car.
          if id in down:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
           counter_down.add(id) 

    # # line
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    
    # print(down)
    cv2.line(frame,(282,308),(1004,308),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(280,308),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 


    downwards = (len(counter_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    


    # cv2.line(frame,(282,308),(1004,308),red_color,3)  #  starting cordinates and end of line cordinates
    # cv2.putText(frame,('red line'),(280,308),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 4 cars, 988.0ms
Speed: 38.0ms preprocess, 988.0ms inference, 70.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 766.6ms
Speed: 4.4ms preprocess, 766.6ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 734.7ms
Speed: 2.3ms preprocess, 734.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 traffic light, 783.8ms
Speed: 4.0ms preprocess, 783.8ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 834.4ms
Speed: 15.6ms preprocess, 834.4ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 traffic light, 767.5ms
Speed: 4.4ms preprocess, 767.5ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 772.2ms
Speed: 3.5ms preprocess, 772.2ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [5]:
downwards

25

In [9]:
def time_allocation(cars):
    min = 20
    max = 90
    if(cars<10):
        return min
    elif(cars>40):
        return max
    else:
        return min + (cars*50)/30

In [10]:
print(time_allocation(downwards))

61.666666666666664


In [12]:
results = model.track(source="traffictrim.mp4",show=True, stream=True,save=True)
for r in results:
    boxes = r.boxes


video 1/1 (frame 1/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 673.3ms
video 1/1 (frame 2/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 752.6ms
video 1/1 (frame 3/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 670.5ms
video 1/1 (frame 4/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 689.5ms
video 1/1 (frame 5/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 550.4ms
video 1/1 (frame 6/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 546.8ms
video 1/1 (frame 7/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 566.9ms
video 1/1 (frame 8/1366) d:\study\yolov9\traffictrim.mp4: 384x640 3 cars, 537.2ms
video 1/1 (frame 9/1366) d:\study\yolov9\traffictrim.mp4: 384x640 4 cars, 536.2ms
video 1/1 (frame 10/1366) d:\study\yolov9\traffictrim.mp4: 384x640 4 cars, 577.3ms
video 1/1 (frame 11/1366) d:\study\yolov9\traffictrim.mp4: 384x640 4 cars, 522.6ms
video 1/1 (frame 12/1366) d:\study\yolov9\traffictrim.mp4: 384x640 4 cars, 571.6ms
video 1/1 (f